<a href="https://colab.research.google.com/github/chaithanyatenneti/Cloud_Detection/blob/main/Cloud_Detection.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import torch
from torchvision import datasets, transforms
from torch.utils.data import DataLoader, Dataset
import matplotlib.pyplot as plt
import numpy as np
import os
from PIL import Image

In [2]:
true_folder = 'Data/Train/true'
false_folder = 'Data/Train/false'

In [3]:
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

In [4]:
class ImageFolder(Dataset):
    def __init__(self, root, transform=None, is_true=True):
        self.root = root
        self.transform = transform
        self.is_true = is_true
        self.image_files = [f for f in os.listdir(root) if f.endswith(('.jpg', '.jpeg', '.png'))]

    def __len__(self):
        return len(self.image_files)

    def __getitem__(self, idx):
        img_path = os.path.join(self.root, self.image_files[idx])
        image = Image.open(img_path).convert('RGB')
        if self.transform:
            image = self.transform(image)
        label = 1 if self.is_true else 0  # 1 for true, 0 for false
        return image, label

true_dataset = ImageFolder(true_folder, transform=transform, is_true=True)
false_dataset = ImageFolder(false_folder, transform=transform, is_true=False)

dataset = torch.utils.data.ConcatDataset([true_dataset, false_dataset])


In [5]:
train_size = int(0.8 * len(dataset))
val_size = len(dataset) - train_size
train_dataset, val_dataset = torch.utils.data.random_split(dataset, [train_size, val_size])


train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=32, shuffle=False)

In [6]:
len(train_loader)

28

In [7]:
len(val_loader)

7

In [17]:
from torchvision import models
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
model = models.densenet169(pretrained=True)


c:\ProgramData\Anaconda3\lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
c:\ProgramData\Anaconda3\lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=DenseNet169_Weights.IMAGENET1K_V1`. You can also use `weights=DenseNet169_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/densenet169-b2777c0a.pth" to C:\Users\chait/.cache\torch\hub\checkpoints\densenet169-b2777c0a.pth


  0%|          | 0.00/54.7M [00:00<?, ?B/s]

In [18]:
num_features = model.classifier.in_features
model.classifier = torch.nn.Linear(num_features, 1)

In [19]:
criterion = torch.nn.BCEWithLogitsLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

In [20]:
torch.cuda.empty_cache()
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = model.to(device)
print(model)

DenseNet(
  (features): Sequential(
    (conv0): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
    (norm0): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (relu0): ReLU(inplace=True)
    (pool0): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
    (denseblock1): _DenseBlock(
      (denselayer1): _DenseLayer(
        (norm1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (relu1): ReLU(inplace=True)
        (conv1): Conv2d(64, 128, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (norm2): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (relu2): ReLU(inplace=True)
        (conv2): Conv2d(128, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      )
      (denselayer2): _DenseLayer(
        (norm1): BatchNorm2d(96, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (relu

In [21]:
num_epochs = 25
for epoch in range(num_epochs):
    model.train()
    running_loss = 0.0
    for images, labels in train_loader:
        images = images.to(device)
        labels = labels.unsqueeze(1).float().to(device)

        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        running_loss += loss.item()

    print(f"Epoch {epoch+1}/{num_epochs}, Loss: {running_loss/len(train_loader)}")

Epoch 1/25, Loss: 0.3655778079160622
Epoch 2/25, Loss: 0.2934587850634541
Epoch 3/25, Loss: 0.2626421350453581
Epoch 4/25, Loss: 0.25623560457357336
Epoch 5/25, Loss: 0.26538257407290594
Epoch 6/25, Loss: 0.2397424230085952
Epoch 7/25, Loss: 0.24643002157764776
Epoch 8/25, Loss: 0.18754453331764256
Epoch 9/25, Loss: 0.26292996534279417
Epoch 10/25, Loss: 0.19212690541254623
Epoch 11/25, Loss: 0.1877485421885337
Epoch 12/25, Loss: 0.14212242460676602
Epoch 13/25, Loss: 0.21161976948912656
Epoch 14/25, Loss: 0.20265361413891828
Epoch 15/25, Loss: 0.16733631092522824
Epoch 16/25, Loss: 0.14071174138890846
Epoch 17/25, Loss: 0.14199177634769253
Epoch 18/25, Loss: 0.1111226896090167
Epoch 19/25, Loss: 0.13122189271130733
Epoch 20/25, Loss: 0.12732865821037972
Epoch 21/25, Loss: 0.10539085963474852
Epoch 22/25, Loss: 0.08509907272777387
Epoch 23/25, Loss: 0.06463238843051451
Epoch 24/25, Loss: 0.06474420368405324
Epoch 25/25, Loss: 0.10035404320141035


In [22]:
model.eval()
all_preds = []
all_labels = []
with torch.no_grad():
    for images, labels in val_loader:
        images = images.to(device)
        labels = labels.to(device)
        outputs = model(images)
        preds = torch.sigmoid(outputs) > 0.5
        all_preds.extend(preds.cpu().numpy().flatten())
        all_labels.extend(labels.cpu().numpy())

In [23]:
accuracy = accuracy_score(all_labels, all_preds)
precision = precision_score(all_labels, all_preds)
recall = recall_score(all_labels, all_preds)
f1 = f1_score(all_labels, all_preds)

print(f"Accuracy: {accuracy}")
print(f"Precision: {precision}")
print(f"Recall: {recall}")
print(f"F1 Score: {f1}")

Accuracy: 0.8454545454545455
Precision: 1.0
Recall: 0.7588652482269503
F1 Score: 0.8629032258064516


In [24]:
test_folder = 'Data/Test'

test_dataset = ImageFolder(test_folder, transform=transform)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)

In [25]:
model.eval()
test_preds = []
with torch.no_grad():
    for images, _ in test_loader:
        images = images.to(device)
        outputs = model(images)
        preds = torch.sigmoid(outputs) > 0.5
        test_preds.extend(preds.cpu().numpy().flatten())